In [ ]:
!pip install transformers huggingface_hub datasets torch evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `save_the_datasets` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-aut

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq, DataCollatorWithPadding

from datasets import load_dataset

import evaluate
import numpy as np

In [ ]:
model_names = ['deep-learning-analytics/GrammarCorrector', 'NlpHUST/t5-en-vi-small', 'NlpHUST/t5-vi-en-small']
metric = evaluate.load("sacrebleu")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # Giải mã các dự đoán thành chuỗi văn bản
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Thay thế giá trị -100 trong labels (được dùng khi padding) bằng pad token id của tokenizer
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Tiền xử lý: loại bỏ khoảng trắng thừa
    decoded_preds = [pred.strip() for pred in decoded_preds]
    # Metric BLEU yêu cầu references là danh sách các danh sách (mỗi câu có thể có nhiều bản dịch tham chiếu)
    decoded_labels = [[label.strip()] for label in decoded_labels]

    # Tính toán BLEU score
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    # Trả về kết quả dưới dạng dictionary
    return {"bleu": result["score"]}

In [ ]:
def getModelAndData(model_name):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)

    # 1. Đóng băng toàn bộ encoder
    for param in model.encoder.parameters():
        param.requires_grad = False

    # 2. Đóng băng các tham số chung (nếu bạn không muốn fine-tune các embedding)
    for param in model.shared.parameters():
        param.requires_grad = False

    # 3. Trong phần decoder, đóng băng tất cả các block ngoại trừ block cuối cùng.
    #    Với T5, phần decoder được lưu trong model.decoder.block là một ModuleList.
    num_decoder_blocks = len(model.decoder.block)
    for idx, block in enumerate(model.decoder.block):
        if idx < num_decoder_blocks - 1:  # Đóng băng các block từ 0 đến block thứ (num_decoder_blocks - 2)
            for param in block.parameters():
                param.requires_grad = False
        else:
            # Với block cuối cùng, giữ nguyên (hoặc bạn có thể chọn chỉ fine-tune một phần của block cuối)
            for param in block.parameters():
                param.requires_grad = True

    return tokenizer, model

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def get_filenames_from_path(path):
  filenames = []
  for root, _, files in os.walk(path):
    for file in files:
      filenames.append(os.path.join(root, file))
  return filenames

translate_data = get_filenames_from_path('/content/drive/MyDrive/CDIO3_Dataset/Translation')
grammar_data = get_filenames_from_path('/content/drive/MyDrive/CDIO3_Dataset/Grammarly')

In [ ]:
import random

def select_and_split_dataset(dataset_list, seed=42):
    # Kiểm tra danh sách có rỗng hay không
    if not dataset_list:
        raise ValueError("Danh sách dataset rỗng!")

    # Chọn ngẫu nhiên một file từ danh sách
    file = random.choice(dataset_list)
    selected_file = {
        'train': file,
    }
    # Xóa file đã chọn khỏi danh sách
    dataset_list.remove(file)
    print(f"Đang sử dụng file: {selected_file}")

    # Load dataset từ file CSV đã chọn
    ds = load_dataset("csv", data_files=selected_file, delimiter=";")["train"]

    # Chia dữ liệu thành 80% train và 20% tạm (temp)
    ds_split = ds.train_test_split(test_size=0.2, shuffle=True, seed=seed)
    ds_train = ds_split["train"]
    ds_tmp = ds_split["test"]

    # Chia ds_tmp thành 50% validation và 50% test (tương đương 10% mỗi)
    ds_val_test = ds_tmp.train_test_split(test_size=0.5, shuffle=True, seed=seed)
    ds_val = ds_val_test["train"]
    ds_test = ds_val_test["test"]

    return {"train": ds_train, "validation": ds_val, "test": ds_test}, dataset_list

In [ ]:
def tokenize_grammar(examples):
  inputs = examples['input']
  targets = examples['output']
  model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

  with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

def tokenize_translate_en(examples):
  inputs = examples['en']
  targets = examples['vi']
  model_inputs = tokenizer(inputs, max_length=512, truncation=True)
  with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=512, truncation=True)
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

def tokenize_translate_vi(examples):
  inputs = examples['vi']
  targets = examples['en']
  model_inputs = tokenizer(inputs, max_length=512, truncation=True)
  with tokenizer.as_target_tokenizer():
      labels = tokenizer(targets, max_length=512, truncation=True)
  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
def push_to_hub(model, tokenizer, model_name):
  model.push_to_hub(model_name)
  tokenizer.push_to_hub(model_name)

# Model grammar

## Get model, tokenize

In [ ]:
tokenizer, model = getModelAndData(model_names[0])

## Preprocess Data

In [ ]:
dataset, grammar_data = select_and_split_dataset(grammar_data)
train_dataset = dataset['train']
test_dataset = dataset['test']
val_dataset = dataset['validation']

Đang sử dụng file: {'train': '/content/drive/MyDrive/CDIO3_Dataset/Grammarly/grammarly_73.csv'}


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['input', 'output'],
    num_rows: 1600
})

In [ ]:
train_dataset = train_dataset.map(tokenize_grammar, batched=True)
test_dataset = test_dataset.map(tokenize_grammar, batched=True)
val_dataset = val_dataset.map(tokenize_grammar, batched=True)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Training

In [ ]:
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/CDIO3_Dataset/Models/grammar", eval_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-46-73e2e558cac4>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

## Push to hub

In [ ]:
push_to_hub(model, tokenizer, 'truongpvk41081/grammarly_correction_model')

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


# Model en2vi

## Get model and tokenizer

In [ ]:
model, tokenizer = getModelAndData(model_names[1])

tokenizer_config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/572 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

## Preprocess Data

In [ ]:
dataset, translate_data = select_and_split_dataset(translate_data)
train_dataset = dataset['train']
test_dataset = dataset['test']
val_dataset = dataset['validation']

In [ ]:
train_dataset = train_dataset.map(tokenize_translate_en, batched=True)
test_dataset = test_dataset.map(tokenize_translate_en, batched=True)
val_dataset = val_dataset.map(tokenize_translate_en, batched=True)

## Training

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/CDIO3_Dataset/Models/en2vi", eval_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-27-582e3e0c5137>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
push_to_hub(model, tokenizer, 'truongpvk41081/translate-en-2-vi')

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

# Model vi2en

## Get model and tokenizer

In [ ]:
model, tokenizer = getModelAndData(model_names[2])

tokenizer_config.json:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

## Preprocess Data

In [ ]:
dataset, translate_data = select_and_split_dataset(translate_data)
train_dataset = dataset['train']
test_dataset = dataset['test']
val_dataset = dataset['validation']

In [ ]:
train_dataset = train_dataset.map(tokenize_translate_vi, batched=True)
test_dataset = test_dataset.map(tokenize_translate_vi, batched=True)
val_dataset = val_dataset.map(tokenize_translate_vi, batched=True)

## Training

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
training_args = TrainingArguments(output_dir="/content/drive/MyDrive/CDIO3_Dataset/Models/vi2en", eval_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

<ipython-input-27-582e3e0c5137>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
push_to_hub(model, tokenizer, 'truongpvk41081/translate-vi-2-en')

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]